In [16]:
import pandas as pd

import sys
sys.path.append("src")
from merging_with_id import merging, create_id

In [3]:
# load datasets
datasets = {}
years = range(2016, 2024)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='utf-8', dtype = str)
    except:
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='Windows 1251', dtype = str)
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...
success
EIE 2019 Loading...
success
EIE 2020 Loading...
success
EIE 2021 Loading...
success
EIE 2022 Loading...
success
EIE 2023 Loading...
success


In [20]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year

In [21]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if 'pt' in col])

['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregname', 'physptareaname', 'physpttername', 'chemptname', 'chemptregname', 'chemptareaname', 'chempttername', 'bioptname', 'bioptregname', 'bioptareaname', 'biopttername', 'geoptname', 'geoptregname', 'geoptareaname', 'geopttername', 'engptname', 'engptregname', 'engptareaname', 'engpttername', 'frptname', 'frptregname', 'frptareaname', 'frpttername', 'deuptname', 'deuptregname', 'deuptareaname', 'deupttername', 'sptest', 'spteststatus', 'spptname', 'spptregname', 'spptareaname', 'sppttername', 'rusptname', 'rusptregname', 'rusptareaname', 'ruspttername']
['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregna

In [31]:
test_centers_df = pd.DataFrame(columns=['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'])

In [32]:
for year, dataset in datasets.items():
    prefixes = [col[:-6] for col in dataset.columns if 'ptname' in col]
    for prefix in prefixes:
        dataset_help = dataset.loc[:, [prefix + 'ptname', prefix + 'ptregname', prefix + 'ptareaname', prefix + 'pttername', 'year']]
        dataset_help.rename(columns={prefix + 'ptname': 'ptname', prefix + 'ptregname': 'ptregname', prefix + 'ptareaname': 'ptareaname', prefix + 'pttername': 'pttername'}, inplace=True)
        test_centers_df = pd.concat([test_centers_df, dataset_help], ignore_index=True)
        # print(year, prefix, dataset_help.dropna(how = 'all').shape[0])
    
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
22924687,NaN,NaN,NaN,NaN,2021
22924688,NaN,NaN,NaN,NaN,2021
22924689,NaN,NaN,NaN,NaN,2021
22924690,NaN,NaN,NaN,NaN,2021


In [33]:
test_centers_df.columns

Index(['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'], dtype='object')

In [34]:
test_centers_df.drop_duplicates(inplace=True)
# When the paricipant graduated in the previous years 
# the corresponding information about school (everything with 'eo') is NaN.
# We drop these rows.
test_centers_df.dropna(subset=test_centers_df.columns.difference(['year']), 
                      how = 'all', inplace=True)
test_centers_df.reset_index(drop=True, inplace=True)
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021


In [38]:
locations_df = pd.read_csv("location_data/locations.csv")
locations_df

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,26010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,8030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,26020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,26040000,OC26040000000000000


In [39]:
locations_df.rename(columns={'regname':'ptregname', 'areaname':'ptareaname', 'tername':'pttername'}, inplace=True)
locations_df.columns

Index(['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020', 'KATOTTG_2023'], dtype='object')

In [40]:
# add location codifier and aggregate by years
test_centers_df_loc = test_centers_df.merge(locations_df[['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020']], on=['ptregname', 'ptareaname', 'pttername'], how='left')

test_centers_df_loc

,ptname,ptregname,ptareaname,pttername,year,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016,6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016,4810400000
...,...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021,5110137600
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021,5123755300
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021,1210136300
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021,1210136600


In [12]:
test_centers_df = test_centers_df_loc.loc[:, ['ptname', 'year', 'KOATUU_2020']]
test_centers_df

,ptname,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,4810400000
...,...,...
3690,Спеціалізована школа І-ІІІ ступенів №246 м. Ки...,8036600000
3691,"Комунальний опорний навчальний заклад ""Іларіон...",1224855300
3692,Середня загальноосвітня школа №166 м.Києва,8038900000
3693,Великодолинський заклад загальної середньої ос...,5123755300


In [13]:
test_centers_with_edrpou = test_centers_df.merge(schools_edrpou, left_on='ptname', right_on='eoname', how='inner')
print(test_centers_with_edrpou[test_centers_with_edrpou['KOATUU_2020_x'] != test_centers_with_edrpou['KOATUU_2020_y']].shape[0])
test_centers_with_edrpou[test_centers_with_edrpou['KOATUU_2020_x'] != test_centers_with_edrpou['KOATUU_2020_y']]

29


,ptname,KOATUU_2020_x,eoname,KOATUU_2020_y,KOATUU_2020_reg,year,id,EDRPOU
137,"Філія ""Черкаський кооперативний економіко-прав...",7110136400,"Філія ""Черкаський кооперативний економіко-прав...",7110136700,71,[2018],флячеркаськийкооперативнийекономкоправовийколе...,39843664
227,Житомирський торговельно-економічний коледж Ки...,1810136600,Житомирський торговельно-економічний коледж Ки...,1810136300,18,"[2019, 2020]",житомирськийторговельноекономчнийколеджкивсько...,38343774
230,Житомирський торговельно-економічний коледж Ки...,1810136300,Житомирський торговельно-економічний коледж Ки...,1810136600,18,[2018],житомирськийторговельноекономчнийколеджкивсько...,38343774
331,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036300,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036600,12,"[2017, 2018, 2019, 2020, 2021, 2022]",криворзьказагальноосвтняшкола3ступ32криворзька...,26458191
332,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036600,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036300,12,[2016],криворзьказагальноосвтняшкола3ступ32криворзька...,26458191
339,"Державний навчальний заклад ""Ставропігійське в...",4610137500,"Державний навчальний заклад ""Ставропігійське в...",4610136600,46,"[2018, 2019, 2020, 2021, 2022, 2023]",ставропгйськевищепрофесйнеучилищемльвов,5537093
473,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036300,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036600,12,"[2017, 2018, 2019, 2020, 2021, 2022]",криворзьказагальноосвтняшкола3ступ125криворзьк...,33407534
474,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036600,Криворізька загальноосвітня школа І-ІІІ ступен...,1211036300,12,[2016],криворзьказагальноосвтняшкола3ступ125криворзьк...,33407534
523,Криворізький коледж економіки та управління Де...,1211036300,Криворізький коледж економіки та управління Де...,1211036600,12,"[2018, 2019, 2020]",криворзькийколеджекономкитауправлнядержавногов...,193275
582,Криворізька гімназія № 95 Криворізької міської...,1211036600,Криворізька гімназія № 95 Криворізької міської...,1211036300,12,"[2017, 2018, 2019, 2020, 2021, 2022]",криворзькагмназя95криворзькаднпропетровськаобл,33422840


In [14]:
test_centers_with_edrpou[test_centers_with_edrpou['EDRPOU'].notna()]

,ptname,KOATUU_2020_x,eoname,KOATUU_2020_y,KOATUU_2020_reg,year,id,EDRPOU
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2310700000,Мелітопольська загальноосвітня школа І-ІІІ сту...,2310700000,23,"[2016, 2017, 2018, 2019, 2020, 2021]",мелтопольськазагальноосвтняшкола3ступ7мелтопол...,26317154
1,"Красилівський навчально-виховний комплекс ""Гім...",6822710100,"Красилівський навчально-виховний комплекс ""Гім...",6822710100,68,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",красилвськийнавчальновиховнийкомплексгмназятаз...,25890006
2,Першотравенська загальноосвітня школа І-ІІІ ст...,1212600000,Першотравенська загальноосвітня школа І-ІІІ ст...,1212600000,12,"[2016, 2017, 2018, 2019, 2020, 2021]",першотравенськазагальноосвтняшкола3ступ5,26329200
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,73,"[2016, 2017, 2018, 2019, 2020, 2021]",чернвецьказагальноосвтняшкола3ступ27чернвецькао,21431264
4,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,73,"[2016, 2017, 2018, 2019, 2020, 2021]",чернвецьказагальноосвтняшкола3ступ27чернвецькао,21431264
...,...,...,...,...,...,...,...,...
3163,Спеціалізована школа І-ІІІ ступенів №246 м. Ки...,8036600000,Спеціалізована школа І-ІІІ ступенів №246 м. Ки...,8036600000,80,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]",спецалзованашкола3ступ246мкивазпоглибленимвивч...,22875058
3164,"Комунальний опорний навчальний заклад ""Іларіон...",1224855300,"Комунальний опорний навчальний заклад ""Іларіон...",1224855300,12,"[2018, 2019, 2020, 2021]",комунальнийопорнийнавчальнийзакладларонвськаза...,26461922
3165,Середня загальноосвітня школа №166 м.Києва,8038900000,Середня загальноосвітня школа №166 м.Києва,8038900000,80,"[2021, 2022, 2023]",середнязагальноосвтняшкола166мкив,22880183
3166,Великодолинський заклад загальної середньої ос...,5123755300,Великодолинський заклад загальної середньої ос...,5123755300,51,"[2021, 2022]",великодолинськийзакладзагальнасередньаосвти2ве...,34312150


In [19]:
test_centers_df.to_csv('Test_centers.csv', index=False)

In [18]:
create_id(test_centers_df, 'ptname','id')

,ptname,KOATUU_2020,id
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2310700000,мелтопольськазагальноосвтняшкола3ступ7мелтопол...
1,"Красилівський навчально-виховний комплекс ""Гім...",6822710100,красилвськийнавчальновиховнийкомплексгмназятаз...
2,Першотравенська загальноосвітня школа І-ІІІ ст...,1212600000,першотравенськазагальноосвтняшкола3ступ5
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,чернвецьказагальноосвтняшкола3ступ27чернвецькао
4,Первомайська загальноосвітня школа I-III ступе...,4810400000,первомайськазагальноосвтняшкола3ступ4первомайс...
...,...,...,...
3690,Спеціалізована школа І-ІІІ ступенів №246 м. Ки...,8036600000,спецалзованашкола3ступ246мкивазпоглибленимвивч...
3691,"Комунальний опорний навчальний заклад ""Іларіон...",1224855300,комунальнийопорнийнавчальнийзакладларонвськаза...
3692,Середня загальноосвітня школа №166 м.Києва,8038900000,середнязагальноосвтняшкола166мкив
3693,Великодолинський заклад загальної середньої ос...,5123755300,великодолинськийзакладзагальнасередньаосвти2ве...
